In [7]:
from matplotlib import pyplot as plt
from matplotlib import colors as colors
import numpy as np

import sys 
sys.path.append('../')
import Utils.utils as util
from importlib import reload
reload(util)

from sklearn.linear_model import Lasso
from sklearn.multiclass import OneVsRestClassifier

In [8]:
"""
def convert_patch_into_vector(patch):
    h = patch.shape[0]
    #Convert one patch into a column vector for training
    patch = patch.flatten()
    train_y = patch[(h*(h//2)+(h//2))*3:(h*(h//2)+(h//2))*3+3]
    train_x = np.delete(patch, range((h*(h//2)+(h//2))*3, (h*(h//2)+(h//2))*3+3))
    return np.array((train_x, train_y))
"""
    
def convert_patch_into_vector(patch):
    #Convert one patch into a column vector for training
    patch = patch.flatten()

    return np.array(patch)

def convert_vector_into_patch(vector, h):
    #Convert a one column vector into a patch of size h
    return vector.reshape((h,h,7))
    
def convert_patches_into_training_set(patches):
    #Convert a set of patches into a set of vectors
    return np.array(list(map(convert_patch_into_vector, patches))).T

def inpaint_image(noisy_image, dictionary, coord_noisy_pix, h):
    
    for y, x in coord_noisy_pix:
        print("Correction de x : ", x, " et y : ", y)
        label = util.get_patch(x, y, h, noisy_image)
        label = list(convert_patch_into_vector(label))

        print("--- lasso regression ---")
        model = Lasso(alpha=0.1, fit_intercept=True)
        model.fit(dictionary, label)
        
        coef = model.coef_
        
        print("coef shape : ", coef.shape)
        print("dictionary shape : ", dictionary.shape) 
        
        toto = sum(coef).T * dictionary

        newPixel = get_centered_pixel(model.predict(toto), h)

        print("nouveau pixel : ", newPixel)

        noisy_image[x][y] = newPixel
        util.display_im(noisy_image)

def get_centered_pixel(patch, h):
    #Convert one patch into a column vector for training
    
    return patch[(len(patch)//2):(len(patch)//2)+3]
    


In [ ]:
img_matrice = util.read_im("Data/imgdataset/space.jpg")

#Taille des patchs
h=3

print("taille de l'image : ", img_matrice.shape)

#noisy_image = delete_rect(img_matrice, 400, 400, 500, 100)
#display_im(noisy_image)

#display_im(img_matrice)

#util.display_im(util.get_patch(50, 50, 4, img_matrice))

height = img_matrice.shape[0]
width = img_matrice.shape[1]

noisy_image, noise_pixels = util.noise(img_matrice, 0.0005, width, height, h)

patches = util.get_patches(noisy_image, h, height, width)

noisy_patches = patches[0]
clear_patches = patches[1]

util.display_im(noisy_image)
#print("New patch : ", convert_patch_into_vector(clear_patches[5]))

nbnoisy = np.shape(noisy_patches)[0]
nbclear = np.shape(clear_patches)[0]

print("nbnoisy:", nbnoisy, ",   nbclear:", nbclear)


taille de l'image :  (256, 256, 3)


In [ ]:
training = convert_patches_into_training_set(clear_patches)
training.shape

#a, b = zip(*training)
#a = np.array(a)


In [ ]:
len(noise_pixels)
print(noise_pixels)

In [ ]:
inpaint_image(noisy_image, training, noise_pixels, h)